In [2]:

import numpy as np
import os
from math import pi, sin, log
import scipy.integrate

from Sire.Tools import Parameter, resolveParameters
#from Sire.Units import *
from Sire.Tools.OpenMMMD import *

# Constants
v0 = 1660.53907 # A^3, the standard state volume
R = 0.00198720425864083 # kcal mol-1, the molar gas constant

temperature = Parameter("temperature", 25 * celsius, """Simulation temperature""")
cart_dict = {"anchor_points":{"l1":21, "l2":11, "l3":2, "r1":4946, "r2":4944, "r3":4949}, "equilibrium_values":{ "xr_l1_0":-1.61, "yr_l1_0": 4.48, "zr_l1_0": 0.68}, "force_constants":{"k_xr_l1":2.65, "k_yr_l1":11.91, "k_zr_l1":4.14,"k_phi":21.01, "k_theta":21.81, "k_psi":25.24}, "reference_frame_rotation":{"xl_ref":{"xl_ref_xl": -0.0255, "xl_ref_yl":0.0346, "xl_ref_zl":0.9991}, "yl_ref":{"yl_ref_xl": 0.6906, "yl_ref_yl":-0.7220, "yl_ref_zl":0.0426}, "zl_ref":{"zl_ref_xl": 0.7228, "zl_ref_yl":0.6911, "zl_ref_zl":-0.0055}}}

print("### Running Standard state correction calculation on %s ###")

print("###====================Utilised Parameters=====================###")
print(temperature)
print ("###===========================================================###\n")


def numerical_distance_integrand(r, r0, kr):
    """Integrand for harmonic distance restraint. Domain is on [-infinity, infinity], 
    but this will be truncated to [-8RT, 8 RT] for practicality.

    Args:
        r (float): Distance to be integrated, in Angstrom 
        r0 (float): Equilibrium distance, in Angstrom
        kr (float): Force constant, in kcal mol-1 A-2

    Returns:
        float: Value of integrand
    """
    return np.exp(-(kr*(r-r0)**2)/(2*R*T))


def numerical_theta_integrand(theta, theta0, ktheta):
    """Integrand for harmonic angle restraint on theta. Domain is on [0, pi].

    Args:
        theta (float): Angle to be integrated, in radians
        theta0 (float): Equilibrium angle, in radians
        ktheta (float): Force constant, in kcal mol-1 rad-2

    Returns:
        float: Value of integrand
    """
    return sin(theta)*np.exp(-(ktheta*(theta-theta0)**2)/(2*R*T))


def numerical_angle_integrand(phi, phi0, kphi):
    """Integrand for the harmonic angle restraints other than theta. Domain is on [-pi,pi].

    Args:
        phi (float): Angle to be integrated, in radians
        phi0 (float): Equilibrium angle, in radians
        kphi (float): Force constant, in kcal mol-1 rad-2

    Returns:
        float: Value of integrand
    """
    d_phi = abs(phi - phi0)
    d_phi_corrected = min(d_phi, 2*pi-d_phi) # correct for periodic boundaries
    return np.exp(-(kphi*d_phi_corrected**2)/(2*R*T))


# Get Boresch restraint dict in dict form
#boresch_dict = dict(boresch_restraints_dict.val)
T = 298 # K

z_r = 1

# Radial
for coord in ["xr_l1", "yr_l1", "zr_l1"]:
    r0 = cart_dict['equilibrium_values'][f"{coord}_0"] # A
    kr = cart_dict['force_constants'][f'k_{coord}'] # kcal mol-1 A-1
    dist_at_8RT = 4*np.sqrt((R*T)/kr) # Dist. which gives restraint energy = 8 RT
    r_min = r0 - dist_at_8RT
    r_max = r0 + dist_at_8RT
    integrand = lambda r: numerical_distance_integrand(r, r0, kr)
    z_r *= scipy.integrate.quad(integrand, r_min, r_max)[0]
    print(z_r)

# Theta
theta0 = 0
ktheta = cart_dict["force_constants"][f"k_theta"] # kcal mol-1 rad-2
integrand = lambda theta: numerical_theta_integrand(theta, theta0, ktheta)
z_r *= scipy.integrate.quad(integrand, 0, pi)[0]
print(z_r)

for angle in ["phi", "psi"]:
    theta0 = 0
    ktheta = cart_dict["force_constants"][f"k_{angle}"] # kcal mol-1 rad-2
    integrand = lambda theta: numerical_angle_integrand(theta, theta0, ktheta)
    z_r *= scipy.integrate.quad(integrand, -pi, pi)[0]
    print(z_r)



dg = -R*T*log(8*pi**2*v0/z_r)

print(f"Semi-analytical correction for releasing Cartesian restraints = {dg:.2f} kcal mol-1")

### Running Standard state correction calculation on %s ###
###====================Utilised Parameters=====================###
temperature = 25 C
###===========================================================###

1.1848650074954754
0.6622238283247728
0.6277640532564849
0.016891664455276865
0.007108512646697005
0.0027293132648407723
Semi-analytical correction for releasing Cartesian restraints = -10.47 kcal mol-1
